In [1]:
import numpy as np, os, sys
import pandas as pd

input_file = './testing_stepwise.csv'

In [2]:
data = pd.read_csv(input_file, delimiter=',')

In [3]:
###THIS IS FAKE. THIS IS WHAT CURRENT DATA WILL LOOK LIKE FOR TRAINING BUT NOT TESTING. 

column_names = [
    "pct_HR", "pct_O2Sat", "pct_Temp", "pct_SBP", "pct_MAP", "pct_DBP", "pct_Resp", "pct_FiO2", "pct_pH", "pct_PaCO2", "pct_BUN", 
    "pct_Calcium", "pct_Creatinine", "pct_Glucose", "pct_Magnesium", "pct_Potassium", "pct_Hct", "pct_Hgb", "pct_WBC", "pct_Platelets",
    
    "6mean_HR", "6mean_O2Sat", "6mean_Temp", "6mean_SBP", "6mean_MAP", "6mean_DBP", "6mean_Resp", "6mean_FiO2", "6mean_pH", "6mean_PaCO2", "6mean_BUN", 
    "6mean_Calcium", "6mean_Creatinine", "6mean_Glucose", "6mean_Magnesium", "6mean_Potassium", "6mean_Hct", "6mean_Hgb", "6mean_WBC", "6mean_Platelets", 
    
    "6std_HR", "6std_O2Sat", "6std_Temp", "6std_SBP", "6std_MAP", "6std_DBP", "6std_Resp", "6std_FiO2", "6std_pH", "6std_PaCO2", "6std_BUN", 
    "6std_Calcium", "6std_Creatinine", "6std_Glucose", "6std_Magnesium", "6std_Potassium", "6std_Hct", "6std_Hgb", "6std_WBC", "6std_Platelets",
    
    "ICULOS", "HR", "O2Sat", "Temp", "SBP", "MAP", "DBP", "Resp", "FiO2", "pH", "PaCO2", "BUN", 
    "Calcium", "Creatinine", "Glucose", "Magnesium", "Potassium", "Hct", "Hgb", "WBC", "Platelets", 
    "Age", "Sex", "SepsisLabel",
    
]

In [4]:
####for train, need to make big DF with all features
####for test, just need to get the computed features and the last row and append them all. 
### example of how the actual test works. 

num_rows = len(data)
for t in range(num_rows):
    current_data = data[:t+1]

In [5]:
####feed these into feature extraction, no sense in performing mean, std, pctmiss of some features
### drop iculos age sex sepsislabel FROM FEATUER ENGINEERING PART ONLY. add last row back at end 
###change 3 to whatever "t"

feature_data = data.iloc[0:3,1:21]

In [6]:
#defines a 6hr window, i just pulled this from a paper, the window will likely be different for our task 
## as our scoring function is different, did not have time to optimize this. 

def six_hr_window(current_data):
    if current_data.shape[0] < 6:
        return current_data
    else:
        return current_data.iloc[-6:]

In [7]:
pct_fill_temp = 1- ((feature_data.isna().sum()/feature_data.shape[0]))
mean_temp = (six_hr_window(feature_data).mean())
std_temp = (six_hr_window(feature_data).std())
last_temp = (data.iloc[3,:])


####features in order - percentage fill, mean from 6hr window, std from 6hr window, last row

###last_temp adds the last row back, change 3 to whatever "t" 

In [8]:
#labels_df = pd.DataFrame(columns = column_names)

In [9]:
temp_df = pd.concat([pct_fill_temp, mean_temp,std_temp,last_temp],ignore_index = True).to_frame().T

In [10]:
for col in temp_df:
    temp_df.rename(columns={col:column_names[col]},inplace=True)

In [11]:
temp_df

,pct_HR,pct_O2Sat,pct_Temp,pct_SBP,pct_MAP,pct_DBP,pct_Resp,pct_FiO2,pct_pH,pct_PaCO2,...,Glucose,Magnesium,Potassium,Hct,Hgb,WBC,Platelets,Age,Sex,SepsisLabel
0,0.666667,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.0,0.0,0.0,...,NaN,NaN,NaN,27.5,NaN,NaN,NaN,87.8,0.0,0.0


In [ ]:
#####this is what we want each row of the training set to look like